In [9]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict,Annotated,Literal
from pydantic import Field,BaseModel
from langchain.messages import HumanMessage,SystemMessage
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

from langgraph.prebuilt import ToolNode,tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from dotenv import load_dotenv

import requests
import random

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOllama(model='qwen2:7b')

In [5]:
search_tool = DuckDuckGoSearchRun(region="us-en")

@tool
def calculator(first_num: float, second_num: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.
    Supported operations: add, sub, mul, div
    """
    try:
        if operation == "add":
            result = first_num + second_num
        elif operation == "sub":
            result = first_num - second_num
        elif operation == "mul":
            result = first_num * second_num
        elif operation == "div":
            if second_num == 0:
                return {"error": "Division by zero is not allowed"}
            result = first_num / second_num
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        
        return {"first_num": first_num, "second_num": second_num, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}


@tool
def get_stock_price(symbol: str) -> dict:
    """
    Fetch latest stock price for a given symbol (e.g. 'AAPL', 'TSLA') 
    using Alpha Vantage with API key in the URL.
    """
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=10V8A9FTW8J9LTL5"
    r = requests.get(url)
    return r.json()

In [6]:
tools = [get_stock_price, search_tool, calculator]

In [11]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [12]:
def chat_node(state: ChatState):
    """LLM node that may answer or request a tool call."""
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)

In [19]:
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)
graph.add_node("tools", tool_node)

In [20]:
graph.add_edge(START, "chat_node")

graph.add_conditional_edges('chat_node',tools_condition)

graph.add_edge('tools','chat_node')

In [25]:
chatbot=graph.compile()

In [26]:
out = chatbot.invoke({"messages": [HumanMessage(content="First find out the stock price of Apple using get stock price tool then use the calculator tool to find out how much will it take to purchase 50 shares?")]})
print(out["messages"][-1].content)

The error suggests that the calculator tool currently doesn't support the multiplication operation needed for this calculation. However, we can easily calculate this manually: 

To find out how much it would cost to purchase 50 shares at $266.18 per share, you multiply the stock price by the number of shares:

Cost = Number of Shares * Stock Price

So,

Cost = 50 * $266.18

Let's calculate this manually: 

Cost = 50 * 266.18 = $13,309
